# Austin Wayling

## Research question/interests

I have always loved following sports and stats and my favorite is baseball. The movie Moneyball introduced me to the idea of a team using data analysis to compete despite having a substantially smaller budget. Since then players have approached the game through a different lense causing the way the game is played to change. I am interested in looking into trends throughout the 150 year history of baseball and if the current declining offense is a cause for concern or not.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.__version__

In [ ]:
batting = pd.read_csv("../data/raw/Batting.csv")

In [ ]:
batting

In [ ]:
people = pd.read_csv("../data/raw/People.csv")

In [ ]:
pitching = pd.read_csv("../data/raw/Pitching.csv")

In [ ]:
salaries = pd.read_csv("../data/raw/Salaries.csv")

In [ ]:
fielding = pd.read_csv("../data/raw/Fielding.csv")

# Milestone 3

## Task 1 EDA

In [ ]:
batting.shape

In [ ]:
people.shape

In [ ]:
pitching.shape

In [ ]:
fielding.shape

In [ ]:
people.describe()

In [ ]:
batting.describe()

In [ ]:
pitching.describe()

In [ ]:
salaries.describe()

In [ ]:
fielding.describe()

In [ ]:
batting.columns

In [ ]:
salaries.columns

In [ ]:
pitching.columns

In [ ]:
fielding.columns

In [ ]:
people.columns

In [ ]:
sns.set_theme(style="ticks",
              font_scale=1.3, 
             )
sns.displot(batting["AB"], kde = True, bins = 15)
#Lot's of unqualafied players who only had a few AB's

In [ ]:
salaries.isnull().describe()

In [ ]:
#Salaries start in 1985 and ends in 2016
batting_edit = batting.drop(batting[batting["yearID"] < 1985].index | batting[batting["yearID"] >2016].index)

In [ ]:
batting_edit["PA"] = batting_edit["AB"] + batting_edit["BB"] + batting_edit["HBP"] + batting_edit["SH"] + batting_edit["SF"]
batting_edit

In [ ]:
batgroup = batting_edit.groupby(["playerID", "yearID"])
batgroup.head()

In [ ]:
salgroup = salaries.groupby(["playerID", "yearID"])
salgroup.agg(np.size)[salgroup.agg(np.size)["teamID"] > 1]

In [ ]:
salaries.shape

In [ ]:
batting = batting.fillna(0)
#Values from pre 1911 and othr early seasons
#Likely very little change from an accurate value

In [ ]:
batting["PA"] = batting["AB"] + batting["BB"] + batting["HBP"] + batting["SH"] + batting["SF"]
batting

In [ ]:
bat_total = batting.groupby("playerID").sum()
bat_total

In [ ]:
batting["PA"].describe()

In [ ]:
bat_total["G"].describe()

In [ ]:
bat_total["debut"] = batting.groupby("playerID")["yearID"].min()
bat_total["seasons"] = 1 + batting.groupby("playerID")["yearID"].max() - batting.groupby("playerID")["yearID"].min()
bat_total

In [ ]:
#A qualafied hitter must have 3.1 PA's per team game played. I have determined team games played based on the length of their debut season.
bat_total = bat_total.drop(bat_total[(bat_total["PA"] / (70 * bat_total["seasons"])) < 3.1].index & bat_total[bat_total["debut"] < 1884].index)
bat_total = bat_total.drop(bat_total[(bat_total["PA"] / (112 * bat_total["seasons"])) < 3.1].index & bat_total[(bat_total["debut"] < 1890) & (bat_total["debut"] >= 1884)].index)
bat_total = bat_total.drop(bat_total[(bat_total["PA"] / (132 * bat_total["seasons"])) < 3.1].index & bat_total[(bat_total["debut"] < 1903) & (bat_total["debut"] >= 1890)].index)    
bat_total = bat_total.drop(bat_total[(bat_total["PA"] / (154 * bat_total["seasons"])) < 3.1].index & bat_total[(bat_total["debut"] < 1961) & (bat_total["debut"] >= 1903)].index)  
bat_total = bat_total.drop(bat_total[(bat_total["PA"] / (162 * bat_total["seasons"])) < 3.1].index & bat_total[bat_total["debut"] >= 1961].index)                     

In [ ]:
bat_total.describe()  

In [ ]:
bat_total = bat_total.drop(["yearID", "stint"], axis = 1)

In [ ]:
bat_total.describe()

In [ ]:
bat_total.shape

In [ ]:
sns.scatterplot(data = bat_total, x = "debut", y = "SO")
#Notice how K's increase over time as modern players are less concerned with strikeouts than before and games increased

In [ ]:
bat_total.columns

In [ ]:
bat_total["BA"] = bat_total["H"] / bat_total["AB"]
bat_total["OBP"] = (bat_total["H"] + bat_total["BB"] + bat_total["HBP"])/bat_total["PA"]
bat_total["SLG"] = ((bat_total["H"] - bat_total["2B"] - bat_total["3B"] - bat_total["HR"]) + (2 * bat_total["2B"]) + (3 * bat_total["3B"]) + (4 * bat_total["HR"]))
bat_total["OPS"] = bat_total["OBP"] + bat_total["SLG"]
#Rate stats> OPS is considered to be the best for evaluating perfomance, however advanced stats like OPS+ or wRC+ can be a better measure.

In [ ]:
bat_tot2 = bat_total.drop(["IBB", "HBP", "SH", "SF", "GIDP", "seasons", "AB"], axis = 1)

In [ ]:
#sns.pairplot(bat_tot2)

In [ ]:
batcorr = bat_total.corr()

In [ ]:
sns.scatterplot(data = bat_total, x = "SB", y = "CS")

In [ ]:
bat_year = batting.groupby("yearID").sum()
bat_year

In [ ]:
bat_year["BA"] = bat_year["H"] / bat_year["AB"]
bat_year["OBP"] = (bat_year["H"] + bat_year["BB"] + bat_year["HBP"])/bat_year["PA"]
bat_year["SLG"] = ((bat_year["H"] - bat_year["2B"] - bat_year["3B"] - bat_year["HR"]) + (2 * bat_year["2B"]) + (3 * bat_year["3B"]) + (4 * bat_year["HR"])) / bat_year["AB"]
bat_year["OPS"] = bat_year["OBP"] + bat_year["SLG"]
bat_year.describe()

In [ ]:
sns.scatterplot(data = bat_year, x = bat_year.index, y = "BA")
#Note drop after 1900, decline from mid 20's to 70's and decline since 2000.

In [ ]:
pitch_year = pitching.groupby("yearID").sum()
pitch_year.describe()

In [ ]:
pitch_year.columns

In [ ]:
pitch_year = pitch_year.drop(["stint", "BK", "GF", "SH", "SF", "GIDP"], axis = 1)

In [ ]:
pitch_year

In [ ]:
bat_year[:4]

In [ ]:
pitch_year[:4]

In [ ]:
pitch_year.drop(["BAOpp", "ERA"], axis = 1)

In [ ]:
era = (9 * pitch_year["ER"]) / (pitch_year["IPouts"] / 3)
era


In [ ]:
pitch_year["ERA"] = era

In [ ]:
sns.scatterplot(data = pitch_year, x = pitch_year.index, y = "ERA")
#ERA seems in sync with BA however it is not still in decline (rise of stats like OBP may be the cause?)

In [ ]:
sns.scatterplot(data = pitch_year, x = pitch_year.index, y = "CG")
#Complete games decreasing as pitcher usage changes

After exploring the data more I am interested in the trends throughout the history of baseball. The swings between offense and defense over the years is interesting and a current hot topic in baseball. The start of the 2021 season was compared to the deadball era due to historic offensive lows but the trends appear closer to other era's throughout baseball. I want to see if the reactions to declining offense in baseball is truly a cause for concern, or if the media is overexaggerating the problem.

## Task 2/3 Analysis Pipeline and Method Chaining

In [ ]:
batting_clean = (
                pd.read_csv("../data/raw/Batting.csv")
                .fillna(0)
                .assign(
                    PA = lambda x: x["AB"] + x["BB"] + x["HBP"] + x["SH"] + x["SF"]
                )
)
batting_clean

In [ ]:
bat_clean_year = (
                batting_clean.groupby("yearID").sum()
                .drop(["stint", "SH", "SF", "G"], axis = 1)
                .assign(
                    BA = lambda x: x['H'] / x['AB'],
                    OBP = lambda x: (x['H'] + x['BB'] + x['HBP']) / x['PA'],
                    SLG = lambda x: ((x['H'] - x['2B'] - x['3B'] - x['HR']) + 2 * x['2B'] + 3 * x['3B'] + 4 * x['HR']) / x['AB'],
                    OPS = lambda x: x['OBP'] + x["SLG"]
                )
)
bat_clean_year

In [113]:
def batting_load_and_process(file_path):
    batting_clean = (
                pd.read_csv(file_path)
                .fillna(0)
                .assign(
                    PA = lambda x: x["AB"] + x["BB"] + x["HBP"] + x["SH"] + x["SF"]
                )
        )
    bat_clean_year = (
                batting_clean.groupby("yearID").sum()
                .drop(["stint", "SH", "SF", "G"], axis = 1)
                .assign(
                    BA = lambda x: x['H'] / x['AB'],
                    OBP = lambda x: (x['H'] + x['BB'] + x['HBP']) / x['PA'],
                    SLG = lambda x: ((x['H'] - x['2B'] - x['3B'] - x['HR']) + 2 * x['2B'] + 3 * x['3B'] + 4 * x['HR']) / x['AB'],
                    OPS = lambda x: x['OBP'] + x["SLG"]
                )
        )
    return bat_clean_year

In [114]:
batting_load_and_process('../data/raw/Batting.csv')

,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,GIDP,PA,BA,OBP,SLG,OPS
yearID,,,,,,,,,,,,,,,,,,,
1871,10822,2659,3101,434,239,47,1783.0,441.0,123.0,393,175.0,0.0,0.0,74.0,11215.0,0.286546,0.311547,0.383848,0.695395
1872,15665,3390,4467,581,145,37,2132.0,269.0,134.0,263,265.0,0.0,0.0,97.0,15928.0,0.285158,0.296961,0.347846,0.644807
1873,16959,3580,4926,570,211,47,2331.0,314.0,131.0,335,278.0,0.0,0.0,122.0,17294.0,0.290465,0.304210,0.357273,0.661483
1874,19104,3470,5224,634,194,40,2252.0,242.0,97.0,238,357.0,0.0,0.0,107.0,19342.0,0.273451,0.282391,0.333229,0.615619
1875,26833,4234,6812,839,273,40,2710.0,629.0,320.0,249,675.0,0.0,0.0,142.0,27082.0,0.253867,0.260727,0.309954,0.570681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,165561,21744,42276,8254,873,5610,20745.0,2537.0,1001.0,15088,38982.0,932.0,1651.0,3719.0,184539.0,0.255350,0.319797,0.417405,0.737202
2017,165567,22582,42215,8397,795,6105,21558.0,2527.0,934.0,15829,40104.0,970.0,1763.0,3804.0,185252.0,0.254972,0.322841,0.425912,0.748753
2018,165432,21630,41018,8264,847,5585,20606.0,2474.0,958.0,15686,41207.0,929.0,1922.0,3457.0,185098.0,0.247945,0.316730,0.409419,0.726148


In [ ]:
pitching_clean = (
                pd.read_csv('../data/raw/Pitching.csv')
                .fillna(0)
)
pitching_clean

In [ ]:
pitch_clean_year = (
            pitching_clean.groupby('yearID').sum()
            .drop(["stint", "BK", "GF", "SH", "SF", "GIDP", "BAOpp", "ERA", "W", "L", "G", "GS", "BFP"], axis = 1)
            .assign(
                ERA = lambda x: (9 * x['ER']) / (x['IPouts'] / 3)
            )
)
pitch_clean_year

In [112]:
def pitching_load_and_process(file_path):
    pitching_clean = (
                pd.read_csv('../data/raw/Pitching.csv')
                .fillna(0)
        )
    pitch_clean_year = (
            pitching_clean.groupby('yearID').sum()
            .drop(["stint", "BK", "GF", "SH", "SF", "GIDP", "BAOpp", "ERA", "W", "L", "G", "GS", "BFP"], axis = 1)
            .assign(
                ERA = lambda x: (9 * x['ER']) / (x['IPouts'] / 3)
            )
        )
    return pitch_clean_year

In [115]:
pitching_load_and_process('../data/raw/Pitching.csv')

,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,IBB,WP,HBP,R,ERA
yearID,,,,,,,,,,,,,,
1871,231,4,4,6750,3101,1055,47,393,175,0.0,208,0.0,2659,4.220000
1872,327,10,7,9858,4467,1335,37,263,265,0.0,202,0.0,3390,3.656421
1873,362,8,9,10754,4926,1353,47,335,278,0.0,124,0.0,3580,3.396969
1874,439,15,3,12509,5224,1015,40,238,357,0.0,127,0.0,3470,2.190823
1875,621,49,18,18571,6812,1531,40,249,675,0.0,330,0.0,4234,2.225890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,83,36,1276,129919,42276,20160,5610,15088,38982,932.0,1808,1651.0,21744,4.189687
2017,59,27,1179,129771,42215,20947,6105,15829,40104,970.0,1810,1763.0,22582,4.358208
2018,42,19,1244,130467,41018,20051,5585,15686,41207,929.0,1847,1922.0,21630,4.149532


In [ ]:
diff_cols = pitch_clean_year.columns.difference(bat_clean_year.columns)
data_clean = pd.merge(bat_clean_year, pitch_clean_year[diff_cols], on = ['yearID'])
data_clean

In [116]:
def merge_batting_pitching(bat, pitch):
    diff_cols = pitch.columns.difference(bat.columns)
    data_clean = pd.merge(bat, pitch[diff_cols], on = ['yearID'])
    return data_clean

In [117]:
df1 = batting_load_and_process('../data/raw/Batting.csv')
df2 = pitching_load_and_process('../data/raw/Pitching.csv')
merge_batting_pitching(df1, df2)

,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,...,OBP,SLG,OPS,CG,ER,ERA,IPouts,SHO,SV,WP
yearID,,,,,,,,,,,,,,,,,,,,,
1871,10822,2659,3101,434,239,47,1783.0,441.0,123.0,393,...,0.311547,0.383848,0.695395,231,1055,4.220000,6750,4,4,208
1872,15665,3390,4467,581,145,37,2132.0,269.0,134.0,263,...,0.296961,0.347846,0.644807,327,1335,3.656421,9858,10,7,202
1873,16959,3580,4926,570,211,47,2331.0,314.0,131.0,335,...,0.304210,0.357273,0.661483,362,1353,3.396969,10754,8,9,124
1874,19104,3470,5224,634,194,40,2252.0,242.0,97.0,238,...,0.282391,0.333229,0.615619,439,1015,2.190823,12509,15,3,127
1875,26833,4234,6812,839,273,40,2710.0,629.0,320.0,249,...,0.260727,0.309954,0.570681,621,1531,2.225890,18571,49,18,330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,165561,21744,42276,8254,873,5610,20745.0,2537.0,1001.0,15088,...,0.319797,0.417405,0.737202,83,20160,4.189687,129919,36,1276,1808
2017,165567,22582,42215,8397,795,6105,21558.0,2527.0,934.0,15829,...,0.322841,0.425912,0.748753,59,20947,4.358208,129771,27,1179,1810
2018,165432,21630,41018,8264,847,5585,20606.0,2474.0,958.0,15686,...,0.316730,0.409419,0.726148,42,20051,4.149532,130467,19,1244,1847


In [123]:
import projectfunctions1 as p1
df = p1.batting_load_and_process('../data/raw/Batting.csv')
df

,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,GIDP,PA,BA,OBP,SLG,OPS
yearID,,,,,,,,,,,,,,,,,,,
1871,10822,2659,3101,434,239,47,1783.0,441.0,123.0,393,175.0,0.0,0.0,74.0,11215.0,0.286546,0.311547,0.383848,0.695395
1872,15665,3390,4467,581,145,37,2132.0,269.0,134.0,263,265.0,0.0,0.0,97.0,15928.0,0.285158,0.296961,0.347846,0.644807
1873,16959,3580,4926,570,211,47,2331.0,314.0,131.0,335,278.0,0.0,0.0,122.0,17294.0,0.290465,0.304210,0.357273,0.661483
1874,19104,3470,5224,634,194,40,2252.0,242.0,97.0,238,357.0,0.0,0.0,107.0,19342.0,0.273451,0.282391,0.333229,0.615619
1875,26833,4234,6812,839,273,40,2710.0,629.0,320.0,249,675.0,0.0,0.0,142.0,27082.0,0.253867,0.260727,0.309954,0.570681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,165561,21744,42276,8254,873,5610,20745.0,2537.0,1001.0,15088,38982.0,932.0,1651.0,3719.0,184539.0,0.255350,0.319797,0.417405,0.737202
2017,165567,22582,42215,8397,795,6105,21558.0,2527.0,934.0,15829,40104.0,970.0,1763.0,3804.0,185252.0,0.254972,0.322841,0.425912,0.748753
2018,165432,21630,41018,8264,847,5585,20606.0,2474.0,958.0,15686,41207.0,929.0,1922.0,3457.0,185098.0,0.247945,0.316730,0.409419,0.726148
